In [ ]:
import requests
import re
import collections
import json
import urllib.request
nice_print = {}
langs = []
nl = collections.Counter()
rnl = {}
total_lang_count = {}
most_pop_lang = ''


test = "elmiram, maryszmary, lizaku, gvanrossum"
users = test.split(', ')


def choose_your_fighter(users):
    print('Выберете пользователя из списка и введите его ник')
    print(users)
    user = input()
    while user not in users:
        print('нет, вы ник пользователья введите, а не вот это вот всё')
        user = input()
    return user


def repositories(user):
    where = 'https://github.com/' + user + '?tab=repositories'
    plain_page = requests.get(where)
    chunk = re.findall('<div\sclass="d-inline-block\smb-1">.*<div\sclass="f6\stext-gray\smt-2">', plain_page.text, re.DOTALL)
    together = ''.join(chunk)
    smaller_chunks = together.split('mt-2">')
    for i in range(0, len(smaller_chunks) - 1):
        repos = re.findall('<a\shref="/.*/(.*)"\si', smaller_chunks[i])
        descriptions = re.findall('description">\n([A-Za-z]*.*)\n', smaller_chunks[i])
        descriptions = re.sub('\s{2,}', '', ''.join(descriptions))
        d = {(''.join(repos)): (''.join(descriptions))}
        nice_print.update(d)
    return plain_page


def languages(user):
    where = 'https://github.com/' + user + '?tab=repositories'
    plain_page = requests.get(where)
    chunk = re.findall('<div\sclass="d-inline-block\smb-1">.*\n</li>', plain_page.text,
                       re.DOTALL)
    together = ''.join(chunk)
    smaller_chunks = together.split('</li>')

    for i in range(0, len(smaller_chunks) - 1):
        d = {}
        repos = re.findall('<a\shref="/.*/(.*)"\si', smaller_chunks[i])
        language = re.findall('programmingLanguage">\n([A-Za-z]*.*)\n', smaller_chunks[i])
        language = re.sub('\s{2,}', '', ''.join(language))
        langs.append(language)
        d = {(''.join(repos)): (''.join(language))}
        rnl.update(d)
    for l in rnl:
        nl[rnl[l]] += 1
    return nl


def repos_sum(user):
    token = "c9aa34ced7c8b3e2ceb2513b561dcd49f24edbde"
    a = 1
    url = 'https://api.github.com/users/%s/repos?access_token=%s&page=%d&per_page=110' % (user, token, a)
    response = urllib.request.urlopen(url)
    text = response.read().decode('utf-8')  # читаем ответ в строку
    data = json.loads(text)
    rep_sum = 0
    #    print(user, rep_sum)
    while (len(data)) != 0:
        url = 'https://api.github.com/users/%s/repos?access_token=%s&page=%d&per_page=110' % (user, token, a)
        response = urllib.request.urlopen(url)
        text = response.read().decode('utf-8')  # читаем ответ в строку
        data = json.loads(text)
        a += 1
        rep_sum += len(data)
    return rep_sum


def the_most_repos(users):
    user_repos = {repos_sum(i): i for i in users}
    sorted(user_repos.items(), reverse=True)
    the_winner = sorted(user_repos.items(), reverse=True)[1]
    return the_winner


def the_most_popular_lang(users):
    most_pop_lang_num = 0
    for user in users:
        nl = languages(user)
        for l in nl:
            if l not in total_lang_count:
                d = {l: nl[l]}
                total_lang_count.update(d)
            else:
                total_lang_count[l] += nl[l]
    for i in total_lang_count:
        if total_lang_count[i] >= most_pop_lang_num:
            most_pop_lang_num = total_lang_count[i]
            most_pop_lang = i
        else:
            continue

    return most_pop_lang


def github_superstar(users):
    most_followers = 0
    the_most_popular_user = ''
    user_followers = {}
    for user in users:
        where = 'https://github.com/' + user
        plain_page = requests.get(where)
        followers = re.findall('Followers\n\s*<span\sclass="Counter">\n\s*([0-9.k]+)\n', plain_page.text)
        followers = ''.join(followers)
        if 'k' in followers:
            followers = float(re.sub('k', '', str(followers))) * 1000
        d = {user: followers}
        user_followers.update(d)
#        print(user_followers)
    for i in user_followers:
        if float(user_followers[i]) >= most_followers:
            most_followers = float(user_followers[i])
            the_most_popular_user = i
        else:
            continue
    return the_most_popular_user


def nice_output(user, nl, the_winner, most_pop_lang, the_most_popular_user):
    print("Вы выбрали пользователя:", user)
    print()
# список репозиториев
    print ("{0:25} {1:25} ".format('Репозиторий','Описание'))
    for repository, description in nice_print.items():
        print("{0:25} {1:25}".format(repository, description))
    print()
    print("Языки, на котрых пишет пользователь и количество репозиториев, где они использованы")
    print()
    for value, count in nl.most_common():
        print("{0:15} {1:15}".format(value, count))
    print()
    print("Больше всего репозиториев у пользователя:", the_winner)
    print()
    print("Наиболее популярный язык среди пользователей это -  ", most_pop_lang)
    print("Больше всего фолловеров у ", the_most_popular_user)


def main(users):
    user = choose_your_fighter(users)
    repositories(user)
    languages(user)
    plain_page = repositories(user)
    the_most_repos(users)
    the_most_popular_lang(users)
    github_superstar(users)
    nl = languages(user)
    the_winner = the_most_repos(users)
    most_pop_lang = the_most_popular_lang(users)
    the_most_popular_user = github_superstar(users)
    nice_output(user, nl, the_winner, most_pop_lang, the_most_popular_user)
    return users


if __name__ == '__main__':
    main(users)

    